# Pre-class Reading: Day 16 (Nov 04, 2024)<br>Monte Carlo Methods 2
Learning goals
1. Apply Markov Chain Monte Carlo methods to simple systems

## *No self-assessment questions for this reading*

Similar to the previous reading assignment, the ideas presented here will mostly be new so please engage with the entire reading assignment.

## 1. Introduction to the Monte Carlo Markov Chain Method 

Markov Chain Monte Carlo methods are algorithms where the the progression of the state depends only on the previous state vector and not the entire history of the state. In the following sections we will build up to a simple model of an ideal gas with quantized energy states by first starting out with modelling the random walk problem.

The general approach is as follows (it will make much more sense when we look at our examples):

1. **Initialization**: Start from an initial state.
2. **Proposal**: Propose a move to change to a new state based on the current one. The way this proposal happens is specified by a proposal distribution.
3. **Acceptance**: Decide whether to accept the move to the new state or stay with the current one.
4. **Iteration**: Repeat the proposal and acceptance steps many times.

## 2. Random walk for one particle

*The following two sections draw on examples from Ayars textbook: https://freecomputerbooks.com/Computational-Physics-with-Python-by-Eric-Ayars.html*

Brownian motion can model the motion of a particle---such as smoke---in a gas. Random collisions with the gas molecules cause the smoke to undergo a motion described as a random walk. We will look at a one-dimensional random walk, where at each iteration there will be a step in either the positive or negative direction. Notice that this is very similar to our triangular Plinko simulation.

Based on our general approach introduced earlier, this would look as follows:

1. **Initialization**: Start from an initial state.
  * Start the particle at `x = 0`<br><br>
2. **Proposal**: Propose a move to change to a new state based on the current one. The way this proposal happens is specified by a proposal distribution.
  * Randomly determine if the particle will move in the positive or negative direction, with equal probability.<br><br>
3. **Acceptance**: Decide whether to accept the move to the new state or stay with the current one.
  * For this initial model, we will accept all proposed moves. As a reminder, you have a seen a system where some proposed moves are rejected, which was in our rectangular Plinko simulation where we rejected moves that would take the puck beyond the wall.<br><br>
4. **Iteration**: Repeat the proposal and acceptance steps many times.
  * We will iterate 100 times.<br><br>

In [ ]:
# Run me multiple times to see the different ways the system can evolve

N = 100  # number of steps

# Store locations after each step and times
x = [0]
t = range(N)

# The possible choices
dir_choices = [-1, 1]

# random walk
for i in range(1, N):
    step = np.random.choice(dir_choices)
    x_new = x[-1] + step # move one step in the pos/neg direction
    x.append(x_new)
    

plt.plot(t, x)
plt.xlabel('Time Steps')
plt.ylabel('Position')
plt.grid(True)
plt.show()

## 3. Random walk for a collection of particles

We now expand our random walk to 2000 particles and look at how the overall behaviour of this system will evolve over time. As you have seen when you ran the single-particle code above multiple times, some particles will wander off far in the positive or negative directions and others will remain somewhat close to x = 0. You will not be surprised to learn that if we look at this behaviour for multiple particles at once, we will expect the positions of all particles at a specific time to follow a Gaussian distribution. Further to that, we can also expect that the standard deviation of this Gaussian distribution should evolve like the square root of the number of steps taken.

Summarizing our approach for a collection of particles:

1. **Initialization**: Start from an initial state.
  * Start with 2000 particles at `x = 0`<br><br>
2. **Proposal**: Propose a move to change to a new state based on the current one. The way this proposal happens is specified by a proposal distribution.
  * Randomly determine if *each* particle will move in the positive or negative direction, with equal probability.<br><br>
3. **Acceptance**: Decide whether to accept the move to the new state or stay with the current one.
  * Accept all proposed moves (no bounding walls)<br><br>
4. **Iteration**: Repeat the proposal and acceptance steps many times.
  * We will iterate 200 times.<br><br>
  
Work through the code below carefully to make sense of each step. We use two visualizations to summarize the behaviour of the system. The first is a histogram of all 2000 particles in the system after the final step. Notice that the mean of this is approximately zero. Second is a history of how the mean and standard deviation of `x` for all particles evolved over time. Here we see that the mean remains approximately zero and the standard deviation evolves like the square root of the number of time steps. 

In [ ]:
from scipy import optimize

nsteps = 200 # number of steps
nparticles = 2000  # number of particles

# Initial positions of all particles
x = np.zeros(nparticles)

# Times
t = range(nsteps)

# Arrays to store average and standard deviations of x of all particles at each time
x_mean = np.zeros(nsteps)
x_std = np.zeros(nsteps)

# Evolve the system through each time step
for i in range(1,nsteps):
    
    # Propose the next move for all particles at the same time
    step = np.random.choice([-1,1], size=nparticles)
    
    # Accept all proposed moves
    # - Each particle in x has its own random step
    x = x + step
    
    # Characterize the new state of the system
    x_mean[i] = np.mean(x)
    x_std[i] = np.std(x, ddof=1)

# Fit the evolution of x_std to a*std(t)  

# First define the fitting function
def fun (x, a):
    return a * np.sqrt(x)

# And then perform the fit
popt, pcov = optimize.curve_fit(fun, t, x_std)    
    
# Plotting
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Histogram of final positions
axs[0].hist(x, bins=20)
axs[0].set_title(f'Histogram of Particle Positions After {nsteps} Steps')
axs[0].set_xlabel('Position')
axs[0].set_ylabel('Number of Particles')

# Plot of mean and standard deviation over time
axs[1].plot(t, x_mean, label='Mean Position', linestyle=':')
axs[1].plot(t, x_std, label='Position STD')
axs[1].plot(t, fun(t,*popt), label='fit: sqrt(t)', linestyle='--')
axs[1].set_title('Mean and Standard Deviation of Particle Positions Over Time')
axs[1].set_xlabel('Time step')
axs[1].set_ylabel('Position')
axs[1].legend()

plt.tight_layout() 
plt.show()

## 4. One kernel of popcorn: a single particle in a gravitational energy system

The image below represents cooking popcorn inside large wok, where the parabolic shape of the wok results in a gravitational potential that increases as you move away from its center. In our system the wok is being agitated (shaken) with an external energy $kT$, such that the system is constantly imparting energy to the popcorn kernels. Importantly, the step-like shape of the wok results in there being discrete gravitational potential energies that a popcorn kernel can have and we are going to have it do a random walk up and down between these energy levels as it moves to a higher or lower step, respectively. 

When trying to move to a higher step, a kernal won't always make it because sometimes it doesn't have quite enough energy so will return to the step that it came from. The probability that this move will be accepted decreases the further the kernel is away from the center of the walk, as the changes in gravitational potential from one step to the next increase. When trying to move to a lower step, the kernel will always make it, assuming it is not already on the lowest step.

![popcorn](https://i.ibb.co/fd5X3hz/popcorn.png)

Since the shape of the wok is parabolic ($h \sim x^2$), we can model the gravitational potential energy of a popcorn kernel as 

$$E = mgh = mgx^2.$$

We will further simplify the system by working in a system of units where $m = g = 1$ so that we can write the relationship between energy and position as

$$E(x) = x^2.$$

Keep in mind that these positions are discrete, starting at $x = 0$ in the center of the walk, moving to $x = 1$ or $x = -1$  when the kernel has moved up one step, and so on.

Thus we can write the change in potential energy when moving from one step $(x_i)$ to the next $(x_f)$ as

$$dE = x_f^2 - x_i^2.$$

Finally, we introduce the Metropolis probability which tells us how likely it is that a proposed move that would change the energy of the system by an amount $dE$ will be accepted. We won't concern ourselves with how this was derived for now. As discussed above, when the kernel is not at $x=0$ and is trying to move to a lower step---a decrease in energy---this move is always accepted. When the proposed change in energy would increase the energy of the system, it is accepted with the probability

$$p = e^{- dE/kT},$$

where $kT$ is an energy quantity representing how vigorously the wok is being shaken. 

Let's summarize our approach for a one kernel system:

1. **Initialization**: Start with one popcorn kernel in the "ground state", at `x = 0`.
2. **Proposal**: Propose a random move to change the state of the kernel, to the right $(dx=+1)$ or to the left $(dx=-1)$.
3. **Acceptance**: Accept the move according to the Metropolis probability based on the change in gravitational potential energy due to the proposal
4. **Iteration**: Iterate `Nsteps` times

Run the code below and observe how the particle jumps out of the ground state (the center of the wok) onto the next step and then sometimes even higher steps over time, but that it continues to fall back to $x=0$. Try playing around with the agitation energy of the system `kT` (larger and smaller) to see how this impacts the overall behaviour. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to calculate the change in potential energy
# from one position to the next
def delta_potential_energy(xi, xf):
    
    return xf**2 - xi**2

# Initialize the system
Nsteps = 200

# Agitation energy (aka temperature)
kT = 1.

# The initial position of the kernel
x = 0

# Array to store the position evolution over time
x_t = np.zeros(Nsteps)

for t in range(Nsteps):
    
    # Choose the proposed move
    dx = np.random.choice([-1, 1])    
    
    # Calculate the change in potential energy (xi, xf)
    # for the proposed move
    dE = delta_potential_energy( x , x+dx )

    # Acceptance probability 
    # - When dE is negative (moving to lower energy), 
    #   the quantity "-dE/kT" will be positive
    #   and thus np.exp(-dE/T) will be > 1 and the move will 
    #   always be accepted
    # - When dE is positive (moving to higher energy),
    #   the quantity "-dE/kT" will negative
    #   and there is a finite probability we will accept the move
    #   given by np.exp(-dE/kT)
    p = np.exp(-dE / kT)

    # Decide whether to accept the move by comparing our random number
    # [0,1) to the probability calculated above. If the move is not accepted,
    # the particle remains in the same location
    if np.random.rand() < p:
        x += dx
        
    # Store the position at each step    
    x_t[t] = x
    
# Plotting
plt.plot(x_t)
plt.xlabel("Time")
plt.ylabel("Position")
plt.show()

## 5. Many kernels of popcorn

We extend the previous example to a system with multiple kernels (`Nkernels`) and modify our algorithm so that in each step we propose only a single move by first picking one of the kernels at random and then proposing that it move left or right in position.

Notice that for `kT = 1` and `Nkernels = 100`, the system reaches an equilibrium gravitational potential energy somewhere between 500 and 1000 time steps.

Again, try playing around with the temperature, but also the number of kernels. How does varying these parameters impact the equilibrium energy and how long it takes to get to that equilibrium?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to calculate the change in potential energy
def delta_potential_energy(xi, xf):

    return xf**2 - xi**2


# Initialize the system
Nsteps = 5000
Nkernels = 100

kT = 1.

# Store the initial positions of all kernels
x = np.zeros(Nkernels)

# Store the evolution of the system over time
x_ave = np.zeros(Nsteps)
x_std = np.zeros(Nsteps)
E_t = np.zeros(Nsteps)

for t in range(Nsteps):
    
    # Choose which kernel will have the proposed move
    kernel = np.random.randint(Nkernels)
    
    # Choose the proposed move
    dx = np.random.choice([-1, 1])    
    
    # Calculate the change in potential energy (xi, xf)
    dE = delta_potential_energy( x[kernel] , x[kernel]+dx )

    # Acceptance probability 
    p = np.exp(-dE / kT)

    # Accept or reject the move
    if np.random.rand() < p:
        x[kernel] += dx
        
    x_ave[t] = x.mean()
    x_std[t] = x.std(ddof=1)
    E_t[t] = np.sum(x**2)
    
# Plotting
fig, axs = plt.subplots(3, 1, figsize=(10, 12))

# Histogram of final positions
axs[0].hist(x, bins=np.linspace(-6,6, 13))
axs[0].set_title(f'Histogram of final popcorn kernel positions after {Nsteps} steps')
axs[0].set_xlabel('Position')
axs[0].set_ylabel('Number of Particles')

# Plot of mean and standard deviation over time
axs[1].plot(x_ave, label='Mean Position')
axs[1].plot(x_std, label='Position STD')
axs[1].set_title('Mean and Standard Deviation of Particle Positions Over Time')
axs[1].set_xlabel('Time step')
axs[1].set_ylabel('Position')
axs[1].legend()

# Plot of graviational potential energy in the system over time
axs[2].plot(E_t, label='Total graviational potential energy')
axs[2].set_title('Total graviational potential energy of the system')
axs[2].set_xlabel('Time step')
axs[2].set_ylabel('Energy')
axs[2].legend()

plt.tight_layout() 
plt.show()

## *Submitting this reading assignment*
Before submitting your work, please ensure you have worked carefully through all the cells. Afterward choose: File >> Save_and_Export_Notebook_As >> HTML. This will download an HTML version of your notebook to your computer which you can upload to Canvas